### Работа с S3

In [1]:
import deeplake

/Users/tainella/opt/anaconda3/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.2.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


### Загрузка уже существующего датасета

На S3 должна лежать папка именно с DeepLake датасетом

In [2]:
deeplake.load('s3://test/fer2013', creds = {
   'aws_access_key_id': 'abc', 
   'aws_secret_access_key': 'xyz', 
   #'aws_session_token': '123', # Optional
   'endpoint_url': 'http://localhost:8888'
})

DatasetHandlerError: A Deep Lake dataset does not exist at the given path (s3://test/fer2013). Check the path provided or in case you want to create a new dataset, use deeplake.empty().

Теперь корректный датасет:

In [12]:
ds = deeplake.load('s3://test/deeplake_fer/deeplake_fer', creds = {
   'aws_access_key_id': 'abc', 
   'aws_secret_access_key': 'xyz', 
   #'aws_session_token': '123', # Optional
   'endpoint_url': 'http://localhost:8888'
})

s3://test/deeplake_fer/deeplake_fer loaded successfully.


In [14]:
ds.summary()

Dataset(path='s3://test/deeplake_fer/deeplake_fer', tensors=['images', 'labels'])

 tensor      htype        shape       dtype  compression
 -------    -------      -------     -------  ------- 
 images      image     (21, 48, 48)   uint8    jpeg   
 labels   class_label    (21, 1)     uint32    None   


### Добавление датасета в S3

In [ ]:
!s3cmd sync {local path to deeplake dataset}  s3://{bucket_name}/